<a href="https://colab.research.google.com/github/rooksandkings/APIs/blob/master/Melchior_Technical_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
!{sys.executable} -m pip install --upgrade setuptools
!{sys.executable} -m pip install -r requirements.txt

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.6.0)
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

path = "/content/drive/My Drive/Colab Notebooks/SpyDaily.csv"
data = pd.read_csv(path)
# Dataset is now stored in a Pandas Dataframe

print("Imported Data:")
data.head()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Imported Data:


,Date,Time,Open,High,Low,Last,Volume,NumberOfTrades,BidVolume,AskVolume
0,2007/10/15,00:00:00,156.27,156.36,153.94,155.01,161115008,0,0,0
1,2007/10/16,00:00:00,154.41,156.48,153.47,153.78,166454496,0,0,0
2,2007/10/17,00:00:00,154.98,155.09,152.47,154.25,216672800,0,0,0
3,2007/10/18,00:00:00,153.45,154.19,153.08,153.69,148447904,0,0,0
4,2007/10/19,00:00:00,153.09,156.48,149.66,149.67,297151008,0,0,0


In [3]:
# Cleaning Data:
# Removing Time, NumberOfTrades, BidVolume, AskVolume from dataframe

if " Time" in list(data.columns):
    data = data.drop(columns = " Time")
    print("Dropped Time Column")
    
if " NumberOfTrades" in list(data.columns):
    data = data.drop(columns = " NumberOfTrades")
    print("Dropped NumberOfTrades Column")
    
if " BidVolume" in list(data.columns):
    data = data.drop(columns = " BidVolume")
    print("Dropped BidVolume Column")
    
if " AskVolume" in list(data.columns):
    data = data.drop(columns = " AskVolume")
    print("Dropped AskVolume Column")

data = data.rename(columns={' Open': 'Open', ' High': 'High', ' Low': 'Low', ' Last': 'Close', ' Volume': 'Volume'})
# list(data.columns)

print("Data after Column Rename / Drop of unneeded Columns")    
data.head()

Dropped Time Column
Dropped NumberOfTrades Column
Dropped BidVolume Column
Dropped AskVolume Column
Data after Column Rename / Drop of unneeded Columns


,Date,Open,High,Low,Close,Volume
0,2007/10/15,156.27,156.36,153.94,155.01,161115008
1,2007/10/16,154.41,156.48,153.47,153.78,166454496
2,2007/10/17,154.98,155.09,152.47,154.25,216672800
3,2007/10/18,153.45,154.19,153.08,153.69,148447904
4,2007/10/19,153.09,156.48,149.66,149.67,297151008


In [18]:
import scipy.stats as st

# Inputs for Keras

# Settings for Local Window Variables
ShWindow = 5
LgWindow = 25
ZscoreWindow = 25

# (1) Candlestick characteristics:
# Candle Length, Candle Body Length, Body % size of Upper Tail, Body % size of Lower Tail, Upper Tail % of Candle Length, 
# Lower Tail % of Candle Length, Body Size Stdev from mean 5, 25 day, Upper/Lower Tail size Stdev from mean 5 day, 25 day+
data['LengthofCandle'] = data['High']-data['Low']
data['LengthofCandleBody'] = (data['Open']-data['Close']).abs()

data['LowerTailLength'] = data[["Open", "Close"]].min(axis=1)-data['Low']
data['UpperTailLength'] = data['High'] - data[["Open", "Close"]].max(axis=1)

data['CandleLengthShortStdev'] = data['LengthofCandle'].rolling(min_periods=ShWindow, window=ShWindow).std()
data['CandleLengthLgWindowStdev'] = data['LengthofCandle'].rolling(min_periods=LgWindow, window=LgWindow).std()

data['CandleBodyLengthShortStdev'] = data['LengthofCandleBody'].rolling(min_periods=ShWindow, window=ShWindow).std()
data['CandleBodyLengthLgWindowStdev'] = data['LengthofCandleBody'].rolling(min_periods=LgWindow, window=LgWindow).std()

data['LowerTailShortStDv'] = data['LowerTailLength'].rolling(min_periods=ShWindow, window=ShWindow).std()
data['UpperTailShortStDv'] = data['UpperTailLength'].rolling(min_periods=ShWindow, window=ShWindow).std()

data['LowerTailLgWindowStDv'] = data['LowerTailLength'].rolling(min_periods=LgWindow, window=LgWindow).std()
data['UpperTailLgWindowStDv'] = data['UpperTailLength'].rolling(min_periods=LgWindow, window=LgWindow).std()

# The following was removed due to LoCB, LoC = small number or 0, producing large and infinity ratios. They were replaced with Pvalues
# data['LowerTailPctCandleBody'] = data['LowerTailLength'] / data['LengthofCandleBody']
# data['UpperTailPctCandleBody'] = data['UpperTailLength'] / data['LengthofCandleBody']
# data['LowerTailPctCandleLength'] = data['LowerTailLength'] / data['LengthofCandle']
# data['UpperTailPctCandleLength'] = data['UpperTailLength'] / data['LengthofCandle']


data['CandleLengthLgPValue'] = st.norm.cdf((data['LengthofCandle'] - data['LengthofCandle'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['LengthofCandle'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std())
data['CandleBodyLengthLgPValue'] = st.norm.cdf((data['LengthofCandleBody'] - data['LengthofCandleBody'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['LengthofCandleBody'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std())
data['UpperTailLgPValue'] = st.norm.cdf((data['UpperTailLength'] - data['UpperTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['UpperTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std())
data['LowerTailLgPValue'] = st.norm.cdf((data['LowerTailLength'] - data['LowerTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['LowerTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std())
# This will produce errors due to st.norm.cdf operating on NaN due to LgWindow. The error is ignorable / can be avoided if the LG window is cut before this operation.
# /usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
#   return (a < x) & (x < b)
# /usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
#   return (a < x) & (x < b)
# /usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
#   cond2 = (x >= _b) & cond0

# (2) Moving Average:
# Difference from Short moving average (Open, High, Low, Close, Avg(Open, High, Low, Close))
data['DiffShortO_MA'] = data['Open']-data['Open'].rolling(min_periods=ShWindow, window=ShWindow).mean()
data['DiffShortH_MA'] = data['High']-data['High'].rolling(min_periods=ShWindow, window=ShWindow).mean()
data['DiffShortL_MA'] = data['Low']-data['Low'].rolling(min_periods=ShWindow, window=ShWindow).mean()
data['DiffShortC_MA'] = data['Close']-data['Close'].rolling(min_periods=ShWindow, window=ShWindow).mean()
data['DiffShortA_MA'] = (data['DiffShortO_MA'] + data['DiffShortH_MA'] + data['DiffShortL_MA'] + data['DiffShortC_MA']) / 4
    
# (3) Difference from Long moving average (Open, High, Low, Close, Avg(Open, High, Low, Close))
data['DiffLongO_MA'] = data['Open']-data['Open'].rolling(min_periods=LgWindow, window=LgWindow).mean()
data['DiffLongH_MA'] = data['High']-data['High'].rolling(min_periods=LgWindow, window=LgWindow).mean()
data['DiffLongL_MA'] = data['Low']-data['Low'].rolling(min_periods=LgWindow, window=LgWindow).mean()
data['DiffLongC_MA'] = data['Close']-data['Close'].rolling(min_periods=LgWindow, window=LgWindow).mean()
data['DiffLongA_MA'] = (data['DiffLongO_MA'] + data['DiffLongH_MA'] + data['DiffLongL_MA'] + data['DiffLongC_MA']) / 4

# (4) Slope of 5, 25 day moving average
data['5DSlpOMA'] = (data['Open'] - data['Open'].shift(5)) / 5
data['25DSlpOMA'] = (data['Open'] - data['Open'].shift(25)) / 25
# May need some adjustments here? MA vs Z-Score MA?

# (5) Slope of average of Open, High, Low, Close 3 to 5, 4-6, 5-7, 6-8 days prior
data['3-5DSlpOMA'] = data['Open'].shift(3).diff(periods=2) / 3
data['4-6DSlpOMA'] = data['Open'].shift(4).diff(periods=2) / 3
data['5-7DSlpOMA'] = data['Open'].shift(5).diff(periods=2) / 3
data['6-8DSlpOMA'] = data['Open'].shift(6).diff(periods=2) / 3
# Needs to be checked

# (6) Relative Position:
# High of Day (HOD) Higher than HOD, Open, Close, Low of Day (LOD) Previous 1-5 days?
data['HODvHOD-1'] = data['High'] - data['High'].shift(1)
data['HODvHOD-2'] = data['High'] - data['High'].shift(2)
data['HODvHOD-3'] = data['High'] - data['High'].shift(3)
data['HODvHOD-4'] = data['High'] - data['High'].shift(4)
data['HODvHOD-5'] = data['High'] - data['High'].shift(5)

data['HODvO-1'] = data['High'] - data['Open'].shift(1)
data['HODvO-2'] = data['High'] - data['Open'].shift(2)
data['HODvO-3'] = data['High'] - data['Open'].shift(3)
data['HODvO-4'] = data['High'] - data['Open'].shift(4)
data['HODvO-5'] = data['High'] - data['Open'].shift(5)

data['HODvC-1'] = data['High'] - data['Close'].shift(1)
data['HODvC-2'] = data['High'] - data['Close'].shift(2)
data['HODvC-3'] = data['High'] - data['Close'].shift(3)
data['HODvC-4'] = data['High'] - data['Close'].shift(4)
data['HODvC-5'] = data['High'] - data['Close'].shift(5)

data['HODvLOD-1'] = data['High'] - data['Low'].shift(1)
data['HODvLOD-2'] = data['High'] - data['Low'].shift(2)
data['HODvLOD-3'] = data['High'] - data['Low'].shift(3)
data['HODvLOD-4'] = data['High'] - data['Low'].shift(4)
data['HODvLOD-5'] = data['High'] - data['Low'].shift(5)

# Open Higher than Open, Close, LOD Previous 1-5 days?
data['OvO-1'] = data['Open'] - data['Open'].shift(1)
data['OvO-2'] = data['Open'] - data['Open'].shift(2)
data['OvO-3'] = data['Open'] - data['Open'].shift(3)
data['OvO-4'] = data['Open'] - data['Open'].shift(4)
data['OvO-5'] = data['Open'] - data['Open'].shift(5)

data['OvC-1'] = data['Open'] - data['Close'].shift(1)
data['OvC-2'] = data['Open'] - data['Close'].shift(2)
data['OvC-3'] = data['Open'] - data['Close'].shift(3)
data['OvC-4'] = data['Open'] - data['Close'].shift(4)
data['OvC-5'] = data['Open'] - data['Close'].shift(5)

data['OvLOD-1'] = data['Open'] - data['Low'].shift(1)
data['OvLOD-2'] = data['Open'] - data['Low'].shift(2)
data['OvLOD-3'] = data['Open'] - data['Low'].shift(3)
data['OvLOD-4'] = data['Open'] - data['Low'].shift(4)
data['OvLOD-5'] = data['Open'] - data['Low'].shift(5)

# Close Higher than Close, LOD, previous 1-5 days?
data['CvC-1'] = data['Close'] - data['Close'].shift(1)
data['CvC-2'] = data['Close'] - data['Close'].shift(2)
data['CvC-3'] = data['Close'] - data['Close'].shift(3)
data['CvC-4'] = data['Close'] - data['Close'].shift(4)
data['CvC-5'] = data['Close'] - data['Close'].shift(5)

data['CvC-1'] = data['Close'] - data['Low'].shift(1)
data['CvC-2'] = data['Close'] - data['Low'].shift(2)
data['CvC-3'] = data['Close'] - data['Low'].shift(3)
data['CvC-4'] = data['Close'] - data['Low'].shift(4)
data['CvC-5'] = data['Close'] - data['Low'].shift(5)

# Lod Higher than LOD previous 1-5 days?
data['LODvLOD-1'] = data['Low'] - data['Low'].shift(1)
data['LODvLOD-2'] = data['Low'] - data['Low'].shift(2)
data['LODvLOD-3'] = data['Low'] - data['Low'].shift(3)
data['LODvLOD-4'] = data['Low'] - data['Low'].shift(4)
data['LODvLOD-5'] = data['Low'] - data['Low'].shift(5)

# (7) Engulfing:
# Body, Body and Tail, Tail engulfing previous 1-5 days? Outside bounds of Body, Body and Tail of previous 1-5 Days?
# Day(s) 5 v 4, 5 v 3, 5 v 2, 5 v 1, 4 v 3, 4 v 2, 4 v 1, 3 v 2, 3 v 1, 2 v 1
# & astype(float) to one-hot encode
# One Hot Encoding reversal not needed if reversal can be deduced from former derivation: e.g. If .shift(5) > .shift(4), it can be deduced that .shift(4) < .shift(5) - relevant to engulfing

data['EngulfingBodyvBody5v4'] = ((data['Open'].shift(5) >= data['Open'].shift(4)) & (data['Close'].shift(5) <= data['Close'].shift(4))).astype(int)
data['EngulfingBodyvBody5v3'] = ((data['Open'].shift(5) >= data['Open'].shift(3)) & (data['Close'].shift(5) <= data['Close'].shift(3))).astype(int)
data['EngulfingBodyvBody5v2'] = ((data['Open'].shift(5) >= data['Open'].shift(2)) & (data['Close'].shift(5) <= data['Close'].shift(2))).astype(int)
data['EngulfingBodyvBody5v1'] = ((data['Open'].shift(5) >= data['Open'].shift(1)) & (data['Close'].shift(5) <= data['Close'].shift(1))).astype(int)
data['EngulfingBodyvBody4v3'] = ((data['Open'].shift(4) >= data['Open'].shift(3)) & (data['Close'].shift(4) <= data['Close'].shift(3))).astype(int)
data['EngulfingBodyvBody4v2'] = ((data['Open'].shift(4) >= data['Open'].shift(2)) & (data['Close'].shift(4) <= data['Close'].shift(2))).astype(int)
data['EngulfingBodyvBody4v1'] = ((data['Open'].shift(4) >= data['Open'].shift(1)) & (data['Close'].shift(4) <= data['Close'].shift(1))).astype(int)
data['EngulfingBodyvBody3v2'] = ((data['Open'].shift(3) >= data['Open'].shift(2)) & (data['Close'].shift(3) <= data['Close'].shift(2))).astype(int)
data['EngulfingBodyvBody3v1'] = ((data['Open'].shift(3) >= data['Open'].shift(1)) & (data['Close'].shift(3) <= data['Close'].shift(1))).astype(int)
data['EngulfingBodyvBody2v1'] = ((data['Open'].shift(2) >= data['Open'].shift(1)) & (data['Close'].shift(2) <= data['Close'].shift(1))).astype(int)

data['EngulfingCandleLengthvCandleLength5v4'] = ((data['High'].shift(5) >= data['High'].shift(4)) & (data['Low'].shift(5) <= data['Low'].shift(4))).astype(int)
data['EngulfingCandleLengthvCandleLength5v3'] = ((data['High'].shift(5) >= data['High'].shift(3)) & (data['Low'].shift(5) <= data['Low'].shift(3))).astype(int)
data['EngulfingCandleLengthvCandleLength5v2'] = ((data['High'].shift(5) >= data['High'].shift(2)) & (data['Low'].shift(5) <= data['Low'].shift(2))).astype(int)
data['EngulfingCandleLengthvCandleLength5v4'] = ((data['High'].shift(5) >= data['High'].shift(1)) & (data['Low'].shift(5) <= data['Low'].shift(1))).astype(int)
data['EngulfingCandleLengthvCandleLength4v3'] = ((data['High'].shift(4) >= data['High'].shift(3)) & (data['Low'].shift(4) <= data['Low'].shift(3))).astype(int)
data['EngulfingCandleLengthvCandleLength3v2'] = ((data['High'].shift(4) >= data['High'].shift(2)) & (data['Low'].shift(4) <= data['Low'].shift(2))).astype(int)
data['EngulfingCandleLengthvCandleLength2v1'] = ((data['High'].shift(4) >= data['High'].shift(1)) & (data['Low'].shift(4) <= data['Low'].shift(1))).astype(int)
data['EngulfingCandleLengthvCandleLength3v2'] = ((data['High'].shift(3) >= data['High'].shift(2)) & (data['Low'].shift(3) <= data['Low'].shift(2))).astype(int)
data['EngulfingCandleLengthvCandleLength3v1'] = ((data['High'].shift(3) >= data['High'].shift(1)) & (data['Low'].shift(3) <= data['Low'].shift(1))).astype(int)
data['EngulfingCandleLengthvCandleLength2v1'] = ((data['High'].shift(2) >= data['High'].shift(1)) & (data['Low'].shift(2) <= data['Low'].shift(1))).astype(int)

# (8) Stars or Inverse Stars:
# Tailess if Z score < 1 (9.2%)
Tailless_ZScore_Cap = -1
data['Lower-Tailless'] = ((data['LowerTailLength'] - data['LowerTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['LowerTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std() < Tailless_ZScore_Cap).astype(int)
data['Upper-Tailless'] = ((data['UpperTailLength'] - data['UpperTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['UpperTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std() < Tailless_ZScore_Cap).astype(int)

# Bodyless if Z score < -1 (9.2%)
Bodyless_ZScore_Cap = -1
data['Bodyless'] = ((data['LengthofCandleBody'] - data['LengthofCandleBody'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['LengthofCandleBody'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std() < Bodyless_ZScore_Cap ).astype(int)

# (9) Positioning:
# Gap(s)?
data['GapPrevious1Day'] = ((data['Open'] > data['High'].shift(1)) | (data['Open'] < data['Low'].shift(1))).astype(int)
data['GapPrevious2Day'] = ((data['Open'].shift(1) > data['High'].shift(2)) | (data['Open'].shift(1) < data['Low'].shift(2))).astype(int)
data['GapPrevious3Day'] = ((data['Open'].shift(2) > data['High'].shift(3)) | (data['Open'].shift(2) < data['Low'].shift(3))).astype(int)
data['GapPrevious4Day'] = ((data['Open'].shift(3) > data['High'].shift(4)) | (data['Open'].shift(3) < data['Low'].shift(4))).astype(int)
data['GapPrevious5Day'] = ((data['Open'].shift(4) > data['High'].shift(5)) | (data['Open'].shift(4) < data['Low'].shift(5))).astype(int)

# Close Higher than previous 1-5 days?, Open higher than previous 1-5 days? Low of day Higher than 
data['CloseHigherv1'] = (data['Close'] > data['Close'].shift(1)).astype(int)
data['CloseHigherv2'] = (data['Close'].shift(1) > data['Close'].shift(2)).astype(int)
data['CloseHigherv3'] = (data['Close'].shift(2) > data['Close'].shift(3)).astype(int)
data['CloseHigherv4'] = (data['Close'].shift(3) > data['Close'].shift(4)).astype(int)
data['CloseHigherv5'] = (data['Close'].shift(4) > data['Close'].shift(5)).astype(int)

data['OpenHigherv1'] = (data['Open'] > data['Open'].shift(1)).astype(int)
data['OpenHigherv2'] = (data['Open'].shift(1) > data['Open'].shift(2)).astype(int)
data['OpenHigherv3'] = (data['Open'].shift(2) > data['Open'].shift(3)).astype(int)
data['OpenHigherv4'] = (data['Open'].shift(3) > data['Open'].shift(4)).astype(int)
data['OpenHigherv5'] = (data['Open'].shift(4) > data['Open'].shift(5)).astype(int)

data['LodHigherv1'] = (data['Low'] > data['Low'].shift(1)).astype(int)
data['LodHigherv2'] = (data['Low'].shift(1) > data['Low'].shift(2)).astype(int)
data['LodHigherv3'] = (data['Low'].shift(2) > data['Low'].shift(3)).astype(int)
data['LodHigherv4'] = (data['Low'].shift(3) > data['Low'].shift(4)).astype(int)
data['LodHigherv5'] = (data['Low'].shift(4) > data['Low'].shift(5)).astype(int)

data['HodHigherv1'] = (data['High'] > data['High'].shift(1)).astype(int)
data['HodHigherv2'] = (data['High'].shift(1) > data['High'].shift(2)).astype(int)
data['HodHigherv3'] = (data['High'].shift(2) > data['High'].shift(3)).astype(int)
data['HodHigherv4'] = (data['High'].shift(3) > data['High'].shift(4)).astype(int)
data['HodHigherv5'] = (data['High'].shift(4) > data['High'].shift(5)).astype(int)


# Queries
# data.loc[data['GapPrevious1Day'] == 1]
# data[['Date', 'Open', 'High', 'Low', 'Close', 'LengthofCandle', 'LengthofCandleBody', 'UpperTailPctCandleBody', 'LowerTailPctCandleBody', 'UpperTailPctCandleLength', 'LowerTailPctCandleLength', 'UpperTailPctLowerTail', 'CandleLength5DStdev']].head(30)
data[['CandleLengthLgPValue']].head(30)
# data[['Open']].shift(1).head(50)
# data[['Open']].head(50)
# print(Tailless_ZScore_Cap)
# data.nsmallest(30, 'High-Tailless')
# data.head(30)

# data[['Date', 'Open', 'High', 'Low', 'Close', 'Upper-Tailless', 'Lower-Tailless']].head(50)

# Do we need to note how many times the slope inverts over a rolling time frame?

# Don't Forget to remove 25 days of data from training due to 25 Day SMA NaN!

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


,CandleLengthLgPValue
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [0]:
# Outputs for Keras


In [0]:
# Export Dataframe to CSV for review (Checking where the Z number extremes are)

data.to_csv(r'/content/drive/My Drive/Colab Notebooks/SpyDailyDF.csv')

In [9]:
import scipy.stats as st
st.norm.cdf(-1.64)

0.05050258347410371